In [26]:
pip install mne

In [27]:
pip install pyeeg

In [28]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [29]:
%pip install qgrid

In [30]:
%matplotlib inline
import mne
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})


import os
from os import walk


# Scipy
from scipy.fft import fft, fftfreq
import scipy.signal as signal
import pickle


# For Data Visulisation
import qgrid

from matplotlib import animation
from IPython.display import HTML

import ipywidgets as widgets
from ipywidgets import interact, interact_manual


from mne import event, pick_types
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.preprocessing import (ICA,create_ecg_epochs,create_eog_epochs,compute_proj_ecg,compute_proj_eog)
from mne.decoding import CSP


**Read in Raw Data**

In [31]:
# 1 .Variable initialisation ************************
raw_fs = 250

eeg_chans = ["C3", "Cz", "C4"]
eog_chans = ["EOG:ch01", "EOG:ch02", "EOG:ch03"]
all_chans = eeg_chans + eog_chans
event_types = {0:"left", 1:"right"}


In [32]:
# 2. Epoching Functions
# Get the list of start times and use this to parse the set of epochs, assign labels.
def getDF(epochs, labels, times, chans):
    data_dict = {}
    for i, label in enumerate(labels): 
        # For each epoch get the start time
        start_time = times[i][0]
        if 'start_time' not in data_dict: 
            data_dict['start_time'] = list()
        data_dict['start_time'].append(start_time)
    
        # For each epoch type add
        if 'event_type' not in data_dict:
            data_dict['event_type'] = list()
        data_dict['event_type'].append(label)
        
        # For epoch add to as a column for each channel
        for ch in range(len(chans)): 
            if chans[ch] not in data_dict:
                data_dict[chans[ch]] = list() 
            data_dict[chans[ch]].append(epochs[i][ch])
        
    return pd.DataFrame(data_dict)

# Break raw data down into epoched sections
def getEpochedDF(eeg_df, event_df, trial_duration_ms=4000):
    epochs = []
    epoch_times = []
    labels = []
    # Get all rows which are the start of a trial
    start_df = eeg_df[eeg_df['EventStart'] == 1]
    for i, event_type in enumerate(event_df["EventType"].values): 
        # Get event type label
        labels.append(event_type)
        
        # Get the start/end time of event marker
        start_time = start_df.iloc[i]["time"]
        end_time = int(start_time + trial_duration_ms)
        
        # Save the time interval
        epoch_times.append((start_time, end_time))
        
        # Get the rows between start time, before the end time
        sub_df = eeg_df[(eeg_df['time'] > start_time) & (eeg_df['time'] <= end_time)]
        eeg_dat = []
        
        # For each channel append the values to the eeg_dat list
        for ch in all_chans: 
            eeg_dat.append(sub_df[ch].values)
        
        # For each epoch append the list of data within the time interval in the order of ch1, ch2, ch3 ... chn
        epochs.append(np.array(eeg_dat))

    # Create dataframe from the data extracted previously
    eeg_epoch_df = getDF(epochs, labels, epoch_times, all_chans)
    return eeg_epoch_df

In [33]:
# 3. Pass data into Epoch functions and create large dataframe
# For each subject:
# Training Labels comes from df_event
# Training Data from df_trainings
df_alldata = pd.DataFrame()
path_train = '/gdrive/MyDrive/UNI:3YP/GoogleCollab/resources/data/y_train_only/'
# GPU path_train = '/gdrive/MyDrive/UNI:3YP/GoogleCollab/resources/data/y_train_only'

files_train = os.listdir(path_train)
n_subjects = len(files_train)
print(n_subjects)

# Store a copy of the list of files, for analysing the freq data afterwards + number of
subject_files_array = []
subject_counts_array = []

for subject_file in range(n_subjects):
    # Get the labels
    # df_event = pd.read_csv("/gdrive/MyDrive/UNI:3YP/GoogleCollab/resources/data/y_train_only/"+files_train[subject_file])
    df_event = pd.read_csv("/gdrive/MyDrive/UNI:3YP/GoogleCollab/resources/data/y_train_only/"+files_train[subject_file])
    # NONGPU df_event = pd.read_csv("/content/drive/MyDrive/UNI:3YP/GoogleCollab/resources/data/y_train_only/"+files_train[subject_file])
    # non gpu
    df_training = pd.read_csv("/gdrive/MyDrive/UNI:3YP/GoogleCollab/resources/data/train/"+files_train[subject_file])

    # Just incase it load subjects percuilaraly
    subject_files_array.append(str(files_train[subject_file]))
    print(files_train[subject_file])
    subject_counts_array.append(len(df_event))
    print('Trials Count:' + str(len(df_event)))

    # Get the training data
    # GPU df_training = pd.read_csv("/gdrive/MyDrive/UNI:3YP/GoogleCollab/resources/data/train/"+files_train[subject_file])
    eeg_epoch_df = getEpochedDF(df_training, df_event, trial_duration_ms=4000) 
    df_alldata = pd.concat([df_alldata, eeg_epoch_df])

immutable_df_alldata = df_alldata


3
B0902T.csv
Trials Count:120
B0903T.csv
Trials Count:160
B0901T.csv
Trials Count:120


In [ ]:
print(df_alldata)

In [34]:
print(subject_files_array)

['B0902T.csv', 'B0903T.csv', 'B0901T.csv']


**Read into MNE Format (for CSP)**

In [35]:
# Initialise lists and define the freq band limits
from mne.io.pick import channel_type
%matplotlib inline
np_trial = None
raw_list = []
raw_labels = []

# Make a 2d array using np storing all the combinations of the band filters
freq_bands_limits = np.array([[8,12], [10,14], [12,16], [14,18], [16,20], [18,22], [20,24], [22,26], [24,28], [26,30]], np.int32)

print(freq_bands_limits)
# For each trial


[[ 8 12]
 [10 14]
 [12 16]
 [14 18]
 [16 20]
 [18 22]
 [20 24]
 [22 26]
 [24 28]
 [26 30]]


*TEST BLOCK - NOT NEEDED, REMOVE AFTER DEVELOPMENT*



In [ ]:
"""
df_alldata_csp = df_alldata
# For each subject get the n number of trials 
for subject_file in range(n_subjects):

  no_of_trials = subject_counts_array[subject_file]
  print(no_of_trials)
  df_subject = df_alldata_csp.head(no_of_trials)
  print(df_subject)

  # Remove the rows from the full data frame
  df_alldata_csp = df_alldata_csp.iloc[no_of_trials:]

# Length should be zero by the end
print(len(df_alldata_csp))
"""

120
     start_time  event_type  \
0      223556.0           1   
1      232996.0           0   
2      251268.0           0   
3      259416.0           0   
4      268576.0           1   
..          ...         ...   
115   2329484.0           0   
116   2357540.0           1   
117   2366300.0           0   
118   2375424.0           0   
119   2410748.0           1   

                                                    C3  \
0    [-0.9826810101472496, -1.574731059739071, -1.3...   
1    [-4.324406805523766, -4.208438239108873, -4.20...   
2    [1.8493934538796064, 0.3204394598306249, 1.406...   
3    [0.9765774013885709, 0.8667124437323568, -0.26...   
4    [0.1251239795529106, 1.2359807736324102, 1.058...   
..                                                 ...   
115  [3.9063096055542834, 3.427176317998016, -0.015...   
116  [-3.6713206683451594, -2.627603570611124, -0.6...   
117  [3.851377126726177, 6.085297932402533, 3.81170...   
118  [-3.6865796902418553, -3.2043945983062

**Step 1: Spectral Filtering**

In [36]:
from mne.io.pick import channel_type
%matplotlib inline


df_alldata_csp = df_alldata

# This will store sets of 10 epochs for each subject
all_subjects_epochs = []
# For each subject get the n number of trials 
for subject_file in range(n_subjects):
  print('Currently at Subject: ' + str(subject_file))

  no_of_trials = subject_counts_array[subject_file]
  print(no_of_trials)
  df_subject = df_alldata_csp.head(no_of_trials)
  print(df_subject)

  # Remove the rows from the full data frame
  df_alldata_csp = df_alldata_csp.iloc[no_of_trials:]

  # Switch back to original form of df method below (ease without retesting)
  # df_all data is exactly just a segment of the data now
  df_alldata = df_subject



  # START OF BY SUBJECT FEATURE EXTRACTION (CSP)
  np_trial = None
  raw_list = []
  raw_labels = []

  # Make a 2d array using np storing all the combinations of the band filters
  freq_bands_limits = np.array([[8,12], [10,14], [12,16], [14,18], [16,20], [18,22], [20,24], [22,26], [24,28], [26,30]], np.int32)
  # We will be making 10 sets of epoch features, one for each band
  all_10_epochs = []

  print(df_alldata.shape)

  # For each frequency band prepare the CSP DATA
  for band in freq_bands_limits:
      lc = band[0]
      hc = band[1]
      print('FIR filtering at: '+ str(lc) + '-' + str(hc))

      # Will store the list of raw data
      raw_list = []


      # Loop through all the trials
      for count, trial in enumerate(df_alldata.iterrows()):
          print('ANALYSING TRIAL:' + str(count))
          # For each trial get the data
          eeg_chans = ["C3", "Cz", "C4"]
          # eog_chans = ["EOG:ch01", "EOG:ch02", "EOG:ch03"]
          ch_names = eeg_chans 
          
          # Get trial data containing raw eeg data
          C3Data = df_alldata['C3'][count] 
          CzData = df_alldata['Cz'][count] 
          C4Data = df_alldata['C4'][count]
          # print(C3Data.shape)

          # Combine individual electrode data to numpy array
          np_trial = np.vstack((C3Data, CzData))
          np_trial = np.vstack((np_trial, C4Data))

          # Get Labels and add to raw data object
          event_type = df_alldata['event_type'][count]
          annotations = mne.Annotations(0, 4.0, event_type)  

          # For each trial make an MNE object
          sfreq = 250 
          ch_type = ['eeg']*len(ch_names)
          info = mne.create_info(ch_names = ch_names, ch_types = ch_type, sfreq = sfreq)
          # info = mne.create_info(ch_names = ch_names, ch_types=['eeg'] *3, sfreq = sfreq)
          montage = mne.channels.make_standard_montage('standard_1020')
          info.set_montage(montage)

          # Build a raw MNE object from the array, and annotate
          data = np_trial
          raw = mne.io.RawArray(data, info)
          raw.load_data()

          # Filter the Raw Data with lowfreq/highfreq cut
          raw.filter(lc, hc, fir_design='firwin')

          if event_type == 1:
            desc = str(event_type) + ': right'
            array = [1 ,0, 1]
            print('right')
          else:
            desc = str(event_type) + ': left'
            # Event Type 0 ie left needed to be 2 as 0 did not work
            array = [0, 0, 2]
            print('left')


          # Annotate object
          my_annot = mne.Annotations(onset=[0],  # in seconds
                              duration=[3.996],  # in seconds, too
                              description=[desc])
          raw.set_annotations(my_annot)
          # Assign Labels
          event_id = event_type


          # Fake the stim data
          stim_data = np.zeros((1, len(raw.times)))
          info = mne.create_info(['STI'], raw.info['sfreq'], ['stim'])
          # Add Fake stim channel
          stim_raw = mne.io.RawArray(stim_data, info)
          raw.add_channels([stim_raw], force_update_info=True)

          # Make events by adding events array (this is the label arrray)
          array = np.array(array)
          arr_2d = np.reshape(array, (1, 3))
          raw.add_events(arr_2d)

          # Combine The Raw Data to Make separate events
          raw.load_data()
          raw_list.append(raw)
          # raw.filter(1,3, picks='all')
          # raw.plot()



      # Get the list of raw data for each trial within that freq band and concat
      raw_concat = concatenate_raws(raw_list, preload=True)
      # Save the events of those raw trials
      events = mne.find_events(raw_concat, stim_channel='STI', verbose=False)
      event_dict = {'MI/left': 2, 'MI/right': 1}

      # Create Epochs
      epochs = mne.Epochs(raw_concat, events, event_id=event_dict, picks='eeg', proj=False, reject=None, flat=None, reject_by_annotation=False, baseline=None, preload=True)
      epochs.plot(picks='all')
      # Add data for that band to epochs
      all_10_epochs.append(epochs)

      # Clear for the next freq band
      epochs = None
      raw_concat = None
      events = None

  # Collate sets of 10 epochs (10 for every subject)
  all_subjects_epochs.append(all_10_epochs)





Output hidden; open in https://colab.research.google.com to view.

In [37]:
# Should be equal to the number of subjects
print(len(all_subjects_epochs))

3


**Step 2: Spatial Filtering**
I now have 10 Epoch sets from each spectral filtering band


In [38]:
from mne.preprocessing import ICA
from mne.preprocessing import create_eog_epochs, create_ecg_epochs
from mne import create_info, find_events, Epochs, concatenate_raws, pick_types
from mne.decoding import CSP
from mne import combine_evoked
from mne.minimum_norm import apply_inverse
from mne.datasets.brainstorm import bst_auditory
from mne.io import read_raw_ctf
from mne.epochs import concatenate_epochs
from sklearn.pipeline import Pipeline  # noqa
from sklearn.model_selection import cross_val_score  # noqa
from sklearn.svm import SVC  # noqa
from sklearn.model_selection import ShuffleSplit  # noqa
from mne.decoding import CSP  # noqa
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import mutual_info_classif as MIC

Save the epochs from subject 1 for development speed

In [ ]:
for count, e in enumerate(all_10_epochs):
  fname = str(count) + 'sub8-epo.fif'
  e.save(fname, overwrite=True)


**Processing Epochs to get Optimum Spatial Filters** 



In [39]:
for subject_number in range(n_subjects):
  # Input: Raw data freq band divided epochs
  # Output: Optimum Frequency band per subject, a list of 27 indexs in the range 0-9 referring to a band.

  all_10_epochs = all_subjects_epochs[subject_number]
  all_opt_freq_bands = []

  nfilters = 4

  # Code for extracting Freq filters/bands
  all_result_strings = []
  all_scores = []

  for count, current_epoch in enumerate(all_10_epochs):
    # We have a list of F input vectors and we need to spatially filter these (using the same CSP settings)
    # The input vectors are stored in a list: all_10_epochs
    print("Total Number of Epochs for freq band:" + str(current_epoch.__len__()))
    this_freqband = freq_bands_limits[count]
    this_epoch = all_10_epochs[count]
    print('Computing Results for CSP Vectors determined using FIR filtering intervals:' + str(freq_bands_limits[count]))

    # Prepare Input Data
    X = this_epoch.get_data()
    y = this_epoch.events[:, -1]
    print(y.shape)
    print(X.shape)

    # CSP,  will order the components of CSP in terms of decreasing mutual information
    csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
    csp.fit(X,y)

    # Create Classifiers
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
    svc = SVC(C=1, kernel='linear')

    # reuglarized csp with value 120
    clf = Pipeline([('CSP', csp), ('SVC', svc)])
    scores = cross_val_score(clf, X, y, cv=cv, n_jobs=1)
    result_string = 'Accuracy' + str(freq_bands_limits[count]) + ': ' + str(scores.mean())
    all_result_strings.append(result_string)
    all_scores.append(scores.mean())

  for result in all_result_strings:
    print(result)

  # For this subject get the optimum frequency band
  optimum_freq_band_idx = np.argmax(all_scores)
  # Add the optimum score for this subject
  all_opt_freq_bands.append(optimum_freq_band_idx)

print("The Optimum Bands for this Subject are in order:")
print(all_opt_freq_bands)


Total Number of Epochs for freq band:119
Computing Results for CSP Vectors determined using FIR filtering intervals:[ 8 12]
(119,)
(119, 3, 176)
Computing rank from data with rank=None
    Using tolerance 0.17 (2.2e-16 eps * 3 dim * 2.5e+14  max singular value)
    Estimated rank (mag): 3
    MAG: rank 3 computed from 3 data channels with 0 projectors
Reducing data rank from 3 -> 3
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.18 (2.2e-16 eps * 3 dim * 2.6e+14  max singular value)
    Estimated rank (mag): 3
    MAG: rank 3 computed from 3 data channels with 0 projectors
Reducing data rank from 3 -> 3
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.16 (2.2e-16 eps * 3 dim * 2.3e+14  max singular value)
    Estimated rank (mag): 3
    MAG: rank 3 computed from 3 data channels with 0 projectors
Reducing data rank from 3 -> 3
Estimating covariance using EMPIRICAL
Done.
Com

In [ ]:
%matplotlib inline